<img src="../logo.png" height="200" width="900">

# <center> Сбор данных: грязная работа своими руками </center>

# 1. Как выглядит хранилище мемов

## 1.1. Что мы хотим получить


Итак, мы хотим  написать код, который поможет нам скачать полезные данные с сайта [knowyourmeme.com:](http://knowyourmeme.com)

- **Name** – название мема,
- **Origin_year** – год его создания,
- **Views** – число просмотров,
- **About** – текстовое описание мема,
- ** и многие другие**

После скачивания и чистки данных, можно будет заняться оцениванием моделей. Например, можно попробовать предсказать популярность мема по его параметрам. Но это все позже, а сейчас познакомимся с парой определений:

* **Парсер** — это скрипт, который собирает информацию с сайта
* **Краулер** — это часть парсера, которая переходит по ссылкам
* **Краулинг** — это переход по страницам и ссылкам
* **Скрапинг** — это сбор данных со страниц
* **Парсинг** — это сразу и краулинг и скрапинг

## 1.2.  Что такое HTML

**HTML (HyperText Markup Language)**  — это язык разметки. Он является стандартным для написания сайтов. Команды в таком языке называются **тегами**. Если открыть абсолютно любой сайт, нажать на правую кнопку мышки, а после нажать `View page source`, то перед вами предстанет HTML скелет этого сайта.

Можно увидеть, что HTML-страница это ни что иное, как набор вложенных тегов. Можно заметить, например, следующие теги:

- `<title>` – заголовок страницы
- `<h1>…<h6>` – заголовки разных уровней
- `<p>` – абзац (paragraph)
- `<div>` – выделения фрагмента документа с целью изменения вида содержимого
- `<table>` – прорисовка таблицы
- `<tr>` – разделитель для строк в таблице
- `<td>` – разделитель для столбцов в таблице
- `<b>` – устанавливает жирное начертание шрифта

Обычно команда `<...>` открывает тег, а  `</...>` закрывает его. Все, что находится между этими двумя командами, подчиняется правилу, которое диктует тег. Например, все, что находится между `<p>` и  `</p>` — это отдельный абзац.   

Теги образуют своеобразное дерево с корнем в теге `<html>` и разбивают страницу на разные логические кусочки. У каждого тега могут быть свои потомки (дети) — те теги, которые вложены в него, и свои родители.

Например, HTML-древо страницы может выглядеть вот так:

    <html>
    <head> Заголовок </head>
    <body>
        <div>
            Первый кусок текста со своими свойствами
        </div>
        <div>
            Второй кусок текста
                <b>
                    Третий кусок с выделенным текстом
                </b>
        </div>
        Четвёртый кусок текста        
    </body>
    </html>            
    
    
<img align="center" src="pictures/tree.png" >

Можно работать с этим html как с текстом, а можно как с деревом. Обход этого дерева и есть парсинг веб-страницы. Нам нужно находить нужные нам узлы среди всего этого разнообразия и забирать с них информацию.

Вручную обходить эти деревья неудобно, поэтому есть специальные языки для обхода деревьев.

- [CSS-селектор](https://ru.wikibooks.org/wiki/CSS/%D0%A1%D0%B5%D0%BB%D0%B5%D0%BA%D1%82%D0%BE%D1%80%D1%8B) (это когда мы ищем элемент страницы по паре ключ, значение)
- [XPath](https://ru.wikipedia.org/wiki/XPath) (это когда мы прописываем путь по дереву вот так: /html/body/div[1]/div[3]/div/div[2]/div)
- Различные библиотеки, например, BeautifulSoup для питона. Именно эту библиотеку мы и будем использовать.

## 1.3. Наш первый запрос

Доступ к веб-станицам позволяет получать модуль `requests`. Подгрузим его и ещё пару пакетов.

In [2]:
import requests      # Библиотека для отправки запросов
import numpy as np   # Библиотека для матриц, векторов и линала
import pandas as pd  # Библиотека для табличек
import time          # Библиотека для времени

Для наших благородных исследовательских целей нужно собрать данные по каждому мему с соответствующей ему страницы. Но для начала нужно получить адреса этих страниц. Поэтому открываем основную страницу со всеми выложенными мемами. Выглядит она следующим образом:

<img align="center" src="pictures/memes_main.png">

Отсюда мы и будем собирать ссылки. Сохраним в переменную `page_link` адрес основной страницы и откроем её при помощи библиотеки `requests`.

In [3]:
page_link = 'https://knowyourmeme.com/memes/all/page/2'

In [4]:
response = requests.get(page_link)
response.content[:500]

b"<!DOCTYPE html>\n<html xmlns:fb='https://www.facebook.com/2008/fbml' xmlns='https://www.w3.org/1999/xhtml'>\n<head>\n<meta content='width=1060' name='viewport'>\n<meta content='text/html; charset=utf-8' http-equiv='Content-Type'>\n\n<meta property='og:title' content='All Entries - Page 2' />\n<meta property='og:site_name' content='Know Your Meme' />\n<meta property='og:image' content='https://a.kym-cdn.com/assets/kym-logo-large-2be3f3818691470a0369e154647ca0f0.png' />\n<meta property='og:type' content='a"

Похоже, мы недвусмысленно дали понять серверу, что мы используем python, а именно библиотеку requests версии 2.14.2. Скорее всего, это вызвало у сервера некоторые подозрения относительно наших благих намерений и он решил нас безжалостно отвергнуть. Для сравнения, можно посмотреть, как выглядят request-headers у запроса через браузер:

<img align="center" src="pictures/good_headers.png">

Очевидно, что нашему скромному запросу не тягаться с таким обилием мета-информации, которое передается при запросе из обычного браузера. К счастью, никто нам не мешает притвориться человечными.

Библиотек, которые справляются с такой задачей, существует очень много, мы воспользуемся [`fake-useragent`](https://pypi.python.org/pypi/fake-useragent). При вызове метода из различных кусочков будет генерироваться случайное сочетание операционной системы, спецификаций и версии браузера, которые можно передавать в запрос:

In [5]:
!pip install fake_useragent

In [6]:
from fake_useragent import UserAgent

In [7]:
UserAgent().chrome

'Mozilla/5.0 (Linux; Android 10; K) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Mobile Safari/537.36'

In [8]:
response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
response

<Response [200]>

Cоединение установлено и данные получены.

In [9]:
html = response.content

type(html)

bytes

## 1.4. Beautiful Soup

<img align="center" src="pictures/soup.jpg" height="200" width="200">

Пакет **[bs4 , a.k.a BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/)** был назван в честь стиха про красивый суп из Алисы в стране чудес.

BeautifulSoup — это библиотека, которая из необработанного HTML кода страницы создаёт структурированный массив данных, по которому очень удобно искать необходимые теги, классы, атрибуты, тексты и прочие элементы веб страниц.

> Пакет под названием `BeautifulSoup` — скорее всего, не то, что нам нужно. Это третья версия (*Beautiful Soup 3*), а мы будем использовать четвертую. Нужно будет установить пакет `beautifulsoup4`. Чтобы было совсем весело, при импорте нужно указывать другое название пакета — `bs4`, а импортировать функцию под названием `BeautifulSoup`. В общем, сначала легко запутаться, но эти трудности нужно преодолеть.

```
pip install beautifulsoup4
```

In [10]:
from bs4 import BeautifulSoup

Передадим функции `BeautifulSoup` текст веб-страницы, которую мы скачали выше.

In [11]:
soup = BeautifulSoup(html,'html.parser')

type(soup)

bs4.BeautifulSoup

Посмотрим что лежит внутри переменной `soup`. Невнимательный пользователь, скорее всего, скажет,что ничего вообще не изменилось. Тем не менее, это не так. Теперь мы можем свободно перемещаться по HTML-дереву страницы и искать нужные нам теги.

In [12]:
soup.html.head.title

<title>
All Entries - Page 2 | Know Your Meme
</title>

Можно извлечь из того места, где мы оказались, текст с помощью метода `text`.

In [13]:
soup.html.head.title.text

'\nAll Entries - Page 2 | Know Your Meme\n'

Более того, зная адрес элемента, мы сразу можем найти его. Например, можно сделать это по классу. Следующая команда должна найти элемент, который лежит внутри тега `a` и имеет класс `photo`.

In [14]:
pip install requests-html

  Obtaining dependency information for requests-html from https://files.pythonhosted.org/packages/24/bc/a4380f09bab3a776182578ce6b2771e57259d0d4dbce178205779abdc347/requests_html-0.10.0-py3-none-any.whl.metadata
  Obtaining dependency information for pyquery from https://files.pythonhosted.org/packages/36/b7/f7ccf9e52e2817e1265d3719c600fa4ef33c07de4d5ef0ced3f43ab1cef2/pyquery-2.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for parse from https://files.pythonhosted.org/packages/ce/f0/30fe1494f1910ad3ea40639b13ac48cdb16a8600e8861cbfc2c560661ddf/parse-1.20.1-py2.py3-none-any.whl.metadata
  Obtaining dependency information for pyppeteer>=0.0.14 from https://files.pythonhosted.org/packages/3d/ee/fb2757a38025421fd3844a0ed0a230b78c9c04a66355024436cf3005a70c/pyppeteer-2.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for pyee<12.0.0,>=11.0.0 from https://files.pythonhosted.org/packages/16/cc/5cea8a0a0d3deb90b5a0d39ad1a6a1ccaa40a9ea86d793eb8a49d32a6ed0/pyee-1

In [15]:
from requests_html import HTMLSession

link = 'https://knowyourmeme.com/memes/all/page/2'

session = HTMLSession()
r = session.get(link)

In [16]:
html = r.content

soup = BeautifulSoup(html, 'html.parser')

obj = soup.findAll('a', attrs={'class': 'photo'})
print(obj)

[]


Однако, вопреки нашим ожиданиям, извлечённый объект имеет класс `"photo left"`. Оказывается, `BeautifulSoup4` расценивает аттрибуты `class` как набор отдельных значений, поэтому `"photo left"` для библиотеки равносильно `["photo", "left"]`, а указанное нами значение этого класса `"photo"` входит в этот список. Чтобы избежать такой неприятной ситуации, придется воспользоваться собственной функцией и задать строгое соответствие:

Полученный после поиска объект также обладает структурой bs4. Поэтому можно продолжить искать нужные нам объекты уже в нём. Извлечём ссылку на этот мем. Сделать это можно по атрибуту `href`, в котором лежит наша ссылка.

Обратите внимание, что после преобразований у данных поменялся тип. Теперь он `str`. Это означет, что с ними можно работать как с текстом.

Если несколько элементов на странице обладают указанным адресом, то метод `find` вернёт только самый первый.  Чтобы найти все элементы с таким адресом, нужно использовать метод `findAll`, и на выход будет выдан список. Таким образом, мы можем получить одним поиском сразу все объекты, содержащие ссылки на страницы с мемами.

In [17]:
meme_links = soup.findAll(lambda tag: tag.name == 'a' and tag.get('class') == ['photo'])
meme_links[:3]

[]

Осталось очистить полученный список от лишнего:

In [18]:
meme_links = [link.attrs['href'] for link in meme_links]

In [19]:
meme_links[:10]

[]

In [20]:
len(meme_links)

0

Готово, получили ровно 16 ссылок по числу мемов на одной странице поиска.

Чтобы легче было искать адрес элемента, можно установить для своего браузера специальную утилиту, позволяющую извлекать со страницы нужные теги, например, [selectorgadget.](http://selectorgadget.com/)

Тем не менее, этот путь не подходит для истинного самурая. Есть другой способ — искать теги для каждого нужного нам элемента вручную. Для этого придётся жать правой кнопкой мышки по окну браузера и жать кнопку **Исследовать элемент (Inspect)**. После браузер будет выглядеть так:

<img align="center" src="pictures/memes_inspection.png" >

Полученный html, в котором находится адрес выбранного вами объекта, можно смело копировать в код.

# 2. Собираем ссылки

После того, как мы скачали все ссылки с текущей страницы, нам нужно каким-то образом перейти на соседнюю и начать скачивать ссылки с неё. На сайте это можно сделать пролистав страницу с мемами вниз, javascript-функции подгрузят новые мемы.

Обычно, все параметры, которые мы устанавливаем на сайте для поиска, отражаются на структуре ссылки. Если мы хотим получить первую страницу с мемами, мы должны будем обратиться к сайту по ссылке

                `https://knowyourmeme.com/memes/all/page/1`


Если мы захотим получить вторую страницу, нам придётся заменить номер страницы на 2


                `https://knowyourmeme.com/memes/all/page/2`

Таким образом мы сможем пройтись по всем страницам.

In [21]:
def getPageLinks(page_number):
    """
        Возвращает список ссылок на мемы, полученный с текущей страницы

        page_number: int/string
            номер страницы для парсинга

    """
    # составляем ссылку на страницу поиска
    page_link = 'https://knowyourmeme.com/memes/all/page/{}'.format(page_number)

    # запрашиваем данные по ней
    response = session.get(page_link, headers={'User-Agent': UserAgent().chrome})

    if not response.ok:
      print(response)
      # если сервер нам отказал, вернем пустой лист для текущей страницы
      return []
    else:
      print(response.ok)

    # получаем содержимое страницы
    html = response.content
    soup = BeautifulSoup(html,'html.parser')

    # ищем ссылки на мемы и очищаем их от ненужных тэгов
    meme_links = soup.findAll(lambda tag: tag.name == 'a' and tag.get('class') == ['photo'])
    meme_links = [link.attrs['href'] for link in meme_links]

    return meme_links

In [22]:
meme_links = getPageLinks(1)
meme_links

True


[]

In [23]:
meme_links = getPageLinks(4)
meme_links

True


[]

Отлично, функция работает и теперь мы теоретически можем достать все $18000$ ссылок, для чего нам нужно будет пройтись по $\frac{18000}{16} \approx 1125$ страницам. Прежде чем расстраивать сервер таким количеством запросов, посмотрим, как доставать всю необходимую информацию о конкретном меме.

# 3. Скачиваем информацию об одном меме

По аналогии со ссылками можно извдечь что угодно. Для этого надо сделать несколько шагов:

1. Открываем страничку с мемом
2. Находим любым способом тег для нужной нам информации
3. Вызываем Beautiful Soap
4. ......
5. Profit

Извлечём число просмотров мема.

<img align="center" src="pictures/doge_main.png">

In [30]:
meme_page = 'https://knowyourmeme.com/memes/doge'

response = session.get(meme_page, headers={'User-Agent': UserAgent().chrome})

html = response.content

print(response)

soup = BeautifulSoup(html,'html.parser')

soup

<Response [200]>


<!DOCTYPE html>

<html lang="en" xmlns="https://www.w3.org/1999/xhtml" xmlns:fb="https://www.facebook.com/2008/fbml">
<head>
<title>
Doge | Know Your Meme
</title>
<link href="https://a.kym-cdn.com" rel="preconnect"/>
<link href="https://i.kym-cdn.com" rel="preconnect"/>
<link href="https://ads.blogherads.com" rel="preconnect"/>
<link href="https://a.kym-cdn.com" rel="dns-prefetch"/>
<link href="https://i.kym-cdn.com" rel="dns-prefetch"/>
<link href="https://ads.blogherads.com" rel="dns-prefetch"/>
<meta content="Doge" property="og:title"/>
<meta content="Know Your Meme" property="og:site_name"/>
<meta content="https://i.kym-cdn.com/entries/icons/original/000/013/564/doge.jpg" property="og:image"/>
<meta content="article" property="og:type"/>
<meta content="104675392961482" property="fb:app_id"/>
<meta content="88519108736" property="fb:pages"/>
<meta content="https://www.facebook.com/knowyourmeme" property="article:publisher"/>
<meta content="summary_large_image" name="twitter:card"/>

Посмотрим, как можно извлечь статистику просмотров, комментариев, а также числа загруженных видео и фото, связанных с нашим мемом. Всё это хранится справа вверху под тэгами `"dd"` и с классами  `"views"`, `"videos"`, `"photos"` и `"comments"`

In [31]:
views = soup.find('dd', attrs={'class': 'views'})

views

In [29]:
views = int(views.replace(',', ''))
views

AttributeError: 'NoneType' object has no attribute 'replace'

Напишем функцию для сбора этой статистики.

In [32]:
def getStats(soup, stats):
    """
        Возвращает очищенное число просмотров/коментариев/...

        soup: объект bs4.BeautifulSoup
            представление текущей страницы

        stats: string
            views/videos/photos/comments

    """
    try:
        obj = soup.find('dd', attrs={'class':stats})
        obj = obj.find('a').text
        obj = int(obj.replace(',', ''))
    except:
        obj=None

    return obj

In [33]:
views = getStats(soup, stats='views')
videos = getStats(soup, stats='videos')
photos = getStats(soup, stats='photos')
comments = getStats(soup, stats='comments')

print("Просмотры: {}\nВидео: {}\nФото: {}\nКомментарии: {}".format(views, videos, photos, comments))

Просмотры: None
Видео: None
Фото: None
Комментарии: None


Еще из интересного и исследовательского —  достанем дату и время добавления мема. Если посмотреть на страницу в браузере, можно подумать, что максимум информации, который мы можем извлечь - это число лет, прошедших с момента публикации —  `Added 4 years ago by NovaXP`. Однако мы так просто сдаваться не будем, посмотрим что в html-коде страницы отвечает за эту надпись:

<img align="center" src="pictures/html_time_ago.png" >

Ага! Вот и подробности по дате добавления, с точностью до минуты.

In [34]:
date = soup.find('abbr', attrs={'class':'timeago'}).attrs['title']
date

AttributeError: 'NoneType' object has no attribute 'attrs'

На самом деле, парсеры — дело непредсказуемое. Часто страницы, которые мы парсим, имеют очень неоднородну структуру. Например, если мы парсим мемы, на части страниц может быть указано описание, а на части нет.

Как только код впервые встречается с отсутствием описания, он выдаёт ошибку и останавливается. Чтобы нормально собрать все данные, приходится [прописывать исключения.](https://pythonworld.ru/tipy-dannyx-v-python/isklyucheniya-v-python-konstrukciya-try-except-dlya-obrabotki-isklyuchenij.html) Для этого используют  конструкцию `try - except`

Например, мы хотим извлечь статус мема, для этого найдем окружающие его тэги:

In [ ]:
properties = soup.find('aside', attrs={'class':'left'})

meme_status = properties.dd

In [ ]:
meme_status

<dd>
Confirmed
</dd>

Дальше нужно извлечь из тэгов текст и убрать лишние пробелы.

In [ ]:
meme_status.text.strip()

'Confirmed'

Однако, если неожиданно выяснится, что у мема нет статуса, метод `find` вернёт пустоту. Метод `text`, в свою очередь, не сможет найти в тэгах текст и выдаст ошибку. Чтобы обезопасить себя от таких пустот, можно прописать исключение или `if - else`. Так как в текущем меме статус все-таки есть, нарочно зададим его как пустой объект, чтобы проверить, что ошибка поймается в обоих случаях

In [ ]:
# Делай раз! Ищем статус мема, но не находим его
meme_status = None

# Делай два! Пытаемся извлечь его...

# ... с исключениями
try:
    print(meme_status.text.strip())
# Если возникает ошибка, статус не найден, выдаём пустоту.
except:
    print("Exception")


# ... с проверкой на пустой элемент
if meme_status:
    print(meme_status.text.strip())
else:
    print("Empty")

Exception
Empty


Такой код позволяет обезопасить себя от ошибок во время работы кода.

In [ ]:
properties = soup.find('aside', attrs={'class':'left'})
meme_status = properties.find("dd")

meme_status = "Empty" if not meme_status else meme_status.text.strip()
print(meme_status)

Confirmed


По аналогии можно извлечь всю остальную информацию со страницы.

In [ ]:
def getProperties(soup):
    """
        Возвращает список (tuple) с названием, статусом, типом,
        годом и местом происхождения и тэгами

        soup: объект bs4.BeautifulSoup
            представление текущей страницы

    """
    # название - идёт с самым большим заголовком h1, легко найти
    meme_name = soup.find('section', attrs={'class':'info wide'})
    meme_name = '' if not meme_name else meme_name.find('h1').text.strip()

    # достаём все данные справа от картинки
    properties = soup.find('aside', attrs={'class':'left'})

    # статус идет первым - можно не уточнять класс
    meme_status = None if not properties else properties.find("dd")

    # oneliner, заменяющий try-except: если тэга нет в properties, вернётся объект NoneType,
    # у которого аттрибут text отсутствует, и в этом случае он заменится на пустую строку
    meme_status = "" if not meme_status else meme_status.text.strip()

    # тип мема - обладает уникальным классом
    meme_type = None if not properties else properties.findAll('a', attrs={'class':'entry-type-link'})
    meme_type = "" if not meme_type else ", ".join([t.text for t in meme_type])

    # год происхождения первоисточника можно найти после заголовка Year,
    # находим заголовок, определяем родителя и ищем следущего ребенка - наш раздел
    meme_origin_year = None if not properties else properties.find(string='\nYear\n')
    meme_origin_year = "" if not meme_origin_year else meme_origin_year.parent.find_next().text.strip()

    # сам первоисточник
    meme_origin_place = None if not properties else properties.find('dd', attrs={'class':'entry_origin_link'})
    meme_origin_place = "" if not meme_origin_place else meme_origin_place.text.strip()

    # тэги, связанные с мемом
    meme_tags = None if not properties else properties.find('dl', attrs={'id':'entry_tags'}).find('dd')
    meme_tags = "" if not meme_tags else meme_tags.text.strip()

    return meme_name, meme_status, meme_type, meme_origin_year, meme_origin_place, meme_tags

In [ ]:
getProperties(soup)

('Doge',
 'Confirmed',
 'Animal, Character, Exploitable, Image Macro, Slang',
 '2010',
 'Tumblr',
 'animal, dog, shiba inu, shibe, such doge, super shibe, japanese, tumblr, comic sans, photoshop meme, doges, dogges, reddit, bitcoin, dogecoin, canine, doge meme, atsuko sato, kabosu, doge memes, dogelore, kabosumama')

Свойства мема собрали. Теперь собираем по аналогии его текстовое описание.

In [ ]:
def getText(soup):
    """
        Возвращает текстовые описания мема

        soup: объект bs4.BeautifulSoup
            представление текущей страницы

    """

    # достаём все тексты под картинкой
    body = soup.find('section', attrs={'class':'bodycopy'})

    # раздел about (если он есть), должен идти первым, берем его без уточнения класса
    meme_about = None if not body else body.find('p')
    meme_about = "" if not meme_about else meme_about.text

    # раздел origin можно найти после заголовка Origin или History,
    # находим заголовок, определяем родителя и ищем следущего ребенка - наш раздел
    meme_origin = None if not body else body.find(string='Origin') or body.find(string='History')
    meme_origin = "" if not meme_origin else meme_origin.parent.find_next().text

    # весь остальной текст (если он есть) можно положить в одно текстовое поле

    if body and body.text:
        other_text = body.text.strip().split('\n')[4:]
        other_text = " ".join(other_text).strip()
    else:
        other_text = ""

    return meme_about, meme_origin, other_text

In [ ]:
meme_about, meme_origin, other_text = getText(soup)

print("О чем мем:\n{}\n\nПроисхождение:\n{}\n\nОстальной текст:\n{}...\n"\
      .format(meme_about, meme_origin, other_text[:200]))

О чем мем:
Doge (pronounced /ˈdoʊdʒ/ DOHJ) is a slang term for "dog" that is primarily associated with pictures of Shiba Inus (nicknamed "Shibe") and internal monologue captions on Tumblr. These photos may be photoshopped to change the dog's face or captioned with interior monologues in Comic Sans font. The primary meme and iconography associated with Doge is the Shiba Inu named Kabosu, whose photos taken by her owner Atsuko Sato in early 2010 went viral across the internet, spawning numerous memes and larger trends in the following decades. Starting in 2017, Ironic Doge formats gained prevalence over the original wholesome version as the memetic character continued to evolve.

Происхождение:
The use of the misspelled word "doge" to refer to a dog dates back to June 24th, 2005, when it was mentioned in an episode of Homestar Runner's puppet show. In the episode titled "Biz Cas Fri 1"[2], Homestar calls Strong Bad his "d-o-g-e" while trying to distract him from his work.

Остальной текс

Наконец, создадим функцию, возвращающую всю информацию по текущему мему

In [ ]:
def getMemeData(meme_page):
    """
        Запрашивает данные по странице, возвращает обработанный словарь с данными

        meme_page: string
            ссылка на страницу с мемом

    """

    # запрашиваем данные по ссылке
    response = session.get(meme_page, headers={'User-Agent': UserAgent().chrome})

    print(response)
    if not response.ok:
        # если сервер нам отказал, вернем статус ошибки
        return response.status_code

    # получаем содержимое страницы
    html = response.content
    soup = BeautifulSoup(html,'html.parser')

    # используя ранее написанные функции парсим информацию
    views = getStats(soup=soup, stats='views')
    videos = getStats(soup=soup, stats='videos')
    photos = getStats(soup=soup, stats='photos')
    comments = getStats(soup=soup, stats='comments')

    # дата
    date = soup.find('abbr', attrs={'class':'timeago'})
    date = "" if not date else date.title

    # имя, статус, и т.д.
    meme_name, meme_status, meme_type, meme_origin_year, meme_origin_place, meme_tags = getProperties(soup=soup)

    # текстовые поля
    meme_about, meme_origin, other_text = getText(soup=soup)

    # составляем словарь, в котором будут хранится все полученные и обработанные данные
    data_row = {"name":meme_name, "status":meme_status,
                "type":meme_type, "origin_year":meme_origin_year,
                "origin_place":meme_origin_place,
                "date_added":date, "views":views,
                "videos":videos, "photos":photos, "comments":comments, "tags":meme_tags,
                "about":meme_about, "origin":meme_origin, "other_text":other_text}


    print(data_row)

    return data_row

In [ ]:
data_row = getMemeData('https://knowyourmeme.com/memes/ayumu-kasuga-osaka')

<Response [200]>
{'name': 'Ayumu Kasuga (Osaka)', 'status': 'Confirmed', 'type': 'Character', 'origin_year': '1999', 'origin_place': 'Azumanga Daioh', 'date_added': None, 'views': 107980, 'videos': 1, 'photos': 86, 'comments': 13, 'tags': 'anime, osaka, manga, azumanga daioh, osaker', 'about': 'Ayumu Kasuga is a fictional high school character from manga and anime series Azumanga Daioh. She is often called by the nickname Osaka by her friends, the city Ayumu from which she transferred. She is also known in memes as Osaker after a resurgence in popularity in 2023. She and the show have been the subject of several memes in the 2010s and 2020s.', 'origin': 'Between March and April of 1999, the second chapter of the Azumanga Daioh manga series debuted in Gekkan Komikku Dengeki Daioh magazine.[1] In part 1 of the chapter, Ayumu is introduced as a transfer student from Osaka (shown below).', 'other_text': 'Between March and April of 1999, the second chapter of the Azumanga Daioh manga series

А теперь подготовим табличку, чтобы в неё записывать все данные, добавим в неё первую полученную строку.

In [ ]:
final_df = pd.DataFrame(columns=['name', 'status', 'type', 'origin_year', 'origin_place',
                                 'date_added', 'views', 'videos', 'photos', 'comments',
                                 'tags', 'about', 'origin', 'other_text'])

In [ ]:
final_df._append(data_row, ignore_index=True)

,name,status,type,origin_year,origin_place,date_added,views,videos,photos,comments,tags,about,origin,other_text
0,Ayumu Kasuga (Osaka),Confirmed,Character,1999,Azumanga Daioh,None,107980,1,86,13,"anime, osaka, manga, azumanga daioh, osaker",Ayumu Kasuga is a fictional high school charac...,"Between March and April of 1999, the second ch...","Between March and April of 1999, the second ch..."


In [ ]:
final_df

,name,status,type,origin_year,origin_place,date_added,views,videos,photos,comments,tags,about,origin,other_text


Ещё раз убедимся что всё работает.

In [ ]:
from tqdm import tqdm_notebook

In [ ]:
meme_links = ['https://knowyourmeme.com/memes/people/chuando-tan',
 'https://knowyourmeme.com/memes/steve--2',
 'https://knowyourmeme.com/memes/sakanigadik',
 'https://knowyourmeme.com/memes/subcultures/go-go-loser-ranger-sentai-daishikkaku-ranger-reject',
 'https://knowyourmeme.com/memes/people/hubslife-day-in-the-life-of-a-middle-class-man',
 'https://knowyourmeme.com/memes/foghorn-leghorn-rambling-speech-pattern-parodies',
 'https://knowyourmeme.com/memes/name-100-women-challenge',
 'https://knowyourmeme.com/memes/last-night-we-let-the-x-talk',
 'https://knowyourmeme.com/memes/im-shaking-should-i-wear-this-to-coachella',
 'https://knowyourmeme.com/memes/who-the-fuck-did-your-hair',
 'https://knowyourmeme.com/memes/sites/rtechnicallythetruth-technically-the-truth',
 'https://knowyourmeme.com/memes/events/prime-drink-forever-chemicals-lawsuit',
 'https://knowyourmeme.com/memes/thukuna',
 'https://knowyourmeme.com/memes/kim-jong-uns-friendly-father-song',
 'https://knowyourmeme.com/memes/id-say-the-1830s-but-without-all-the-racists',
 'https://knowyourmeme.com/memes/when-i-die-imma-go-to-snow-bunny-heaven']


In [ ]:
for meme_link in tqdm_notebook(meme_links):
    data_row = getMemeData(meme_link)
    final_df = final_df._append(data_row, ignore_index=True)

<ipython-input-185-f3e28835e4f7>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for meme_link in tqdm_notebook(meme_links):


  0%|          | 0/16 [00:00<?, ?it/s]

<Response [200]>
{'name': '', 'status': '', 'type': '', 'origin_year': '', 'origin_place': '', 'date_added': '', 'views': None, 'videos': None, 'photos': None, 'comments': None, 'tags': '', 'about': '', 'origin': '', 'other_text': ''}
<Response [200]>
{'name': '', 'status': '', 'type': '', 'origin_year': '', 'origin_place': '', 'date_added': '', 'views': None, 'videos': None, 'photos': None, 'comments': None, 'tags': '', 'about': '', 'origin': '', 'other_text': ''}
<Response [200]>
{'name': 'Sakanigadik', 'status': 'Submission', 'type': 'Song', 'origin_year': '2022', 'origin_place': 'YouTube', 'date_added': None, 'views': 195, 'videos': 12, 'photos': 0, 'comments': 2, 'tags': 'sakanigadik, meme, viral video, tiktok, youtube, sak-uh-nigg-adik, slang', 'about': 'Sakanigadik refers to a viral video in which a black man is playing the keyboard and singing the explicit lyric "Suck a nigga dick," which sounds like "sakanigadik" as the words are combined. The music was posted on YouTube in Ap

In [ ]:
final_df

,name,status,type,origin_year,origin_place,date_added,views,videos,photos,comments,tags,about,origin,other_text
0,,,,,,,None,None,None,None,,,,
1,,,,,,,None,None,None,None,,,,
2,,,,,,,None,None,None,None,,,,
3,,,,,,,None,None,None,None,,,,
4,,,,,,,None,None,None,None,,,,
5,,,,,,,None,None,None,None,,,,
6,,,,,,,None,None,None,None,,,,
7,,,,,,,None,None,None,None,,,,
8,,,,,,,None,None,None,None,,,,
9,,,,,,,None,None,None,None,,,,


In [1]:
final_df.shape

NameError: name 'final_df' is not defined

# 4. Итоговый цикл

Осталось написать итоговый цикл. На всякий случай обернём его в `try-except`.

In [ ]:
from tqdm import tqdm_notebook

final_df = pd.DataFrame(columns=['name', 'status', 'type', 'origin_year', 'origin_place',
                                 'date_added', 'views', 'videos', 'photos', 'comments',
                                 'tags', 'about', 'origin', 'other_text'])

for page_number in tqdm_notebook(range(1075), desc='Pages'):

    # собрали ссылки с текущей страницы
    meme_links = getPageLinks(page_number)

    for meme_link in tqdm_notebook(meme_links, desc='Memes', leave=False):

        # иногда с первого раза страничка не прогружается
        for i in range(5):
            try:
                # пытаемся собрать данные
                data_row = getMemeData(meme_link)
                final_df = final_df.append(data_row, ignore_index=True)
                # если всё получилось - выходим из внутреннего цикла
                break
            except:
                # Иначе, пробуем еще несколько раз, пока не закончатся попытки
                print('AHTUNG! parsing once again:', meme_link)
                continue

# 5. Что делать, если сервер разозлился?


* Вы решили собрать себе немного данных
* Сервер не в восторге от ковровой бомбардировки автоматическими запросами
* Error 403, 404, 504, $\ldots$
* Капча, требования зарегистрироваться
* Заботливые сообщения, что с вашего устройства обнаружен подозрительный трафик

<center>
<img src="pictures/doge.jpg" width="450">

### а) быть терпеливым

* Слишком частые запросы раздражают сервер
* Ставьте между ними временные задержки
* Сервер любит временные задержки, так как боится сломаться от перегрузок

In [36]:
import time
time.sleep(3) # и пусть весь мир подождёт 3 секунды

### б) общаться через посредников

<center>
<img src="https://raw.githubusercontent.com/hse-econ-data-science/eds_spring_2020/master/sem05_parsing/image/proxy.jpeg" width="400">

Запрос работал немного подольше, ip адрес сменился. Большая часть прокси-серверов, которые вы найдёте работают плохо. Иногда запрос идёт очень долго и выгоднее сбросить его. Это можно настроить опцией `timeout`.  Например, так если сервер не будет отвечать секунду, код перестанет работать.

In [35]:
import requests
requests.get('http://www.google.com', timeout=1)

<Response [200]>

## в) уходить глубже

<center>
<img src="https://raw.githubusercontent.com/hse-econ-data-science/eds_spring_2020/master/sem05_parsing/image/tor.jpg" width="600">

Можно попытаться обходить злые сервера через тор. Есть несколько способов, но мы про это говорить не будем. Лучше подробно почитать [в нашей статье на Хабре.](https://habr.com/ru/company/ods/blog/346632/)

## Совместить всё?

1. Начните с простых приемов, например с `time.sleep`
2. Пробуйте новые приёмы постепенно
3. Каждый новый приём замедляет скорость работы
4. [Разные продвинутые способы работы с библиотекой requests](http://docs.python-requests.org/en/v0.10.6/user/advanced/)

> Напоследок, хотелось бы сказать пару слов о парсинге вообще и при помощи Тора в частности. Собирать себе данные самостоятельно - это стильно, модно и в принципе интересно, можно получить наборы, которых еще никто никогда не обрабатывал, сделать что-то новое, посмотреть, наконец, на все мемы мира сразу. Однако не стоит забывать, что ограничения, введенные сервером, в том числе баны, появились не просто так, а в целях защиты сайта от DDoS-атак. К чужому труду стоит относится с уважением, и даже если у сервера никакой защиты нет, - это еще не повод неограниченно забрасывать его своими запросами, особенно если это может привести к его отключению - [уголовное наказание](http://sd-company.su/article/security/ddosataka-ugolovnaya-otvetstvennost) никто не отменял. Успешных и безопасных вам исследований!

# Материалы

* [Парсим мемы в python](https://habr.com/ru/company/ods/blog/346632/) - подробная статья на Хабре
* [Продвинутое использование requests](https://2.python-requests.org/en/master/user/advanced/)
* [Репозиторий](https://github.com/DmitrySerg/memology) с исследованием мемов

<img src="pictures/take_all.png" height="200" width="900">  